# Making Sense of API Data with Pandas

Pandas stands for 'Python Data Analysis Library'; it is designed to provide data scientists working in Python with a set of powerful tools to load, transform, and process large-ish data sets. As a result, it has become something of a *de facto* standard for online tutorials and many of the lessons that you can find online will make use of pandas at some point.

You will want to bookmark [the documentation](http://pandas.pydata.org/pandas-docs/stable/) since you will undoubtedly need to refer to it fairly regularly. _Note_: this link is to the most recent, stable release. If you are using an older version of pandas then you'll need to track down the appropriate version from the [home page](http://pandas.pydata.org).

You can always check what version you have installed like this:
```python
import pandas as pd
print pd.__version__
```
*Note*: this approach won't necessarily work with _every_ package, but it will work with many of them. Remember that variables and methods starting and ending with '`__`' are **private** and any interaction with them should be approached very, very carefully.

Anyway, the main elements of pandas with which you interact directly are: 
1. the DataFrame; 
2. the Series;
3. the Index. 

Let's take a look:

In [ ]:
import pandas as pd
help(pd.DataFrame)

On second thought, let's never do that again. Well, at least not _that_ way! You'll have noticed that the help documentation for the DataFrame is not just a bit longer than anything we've seen before, it's massively longer. There's probably quite a lot of intimidating terminology in there too... Right from the start we get things like "Two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns)." 

Here's the thing: in the [last notebook](https://raw.githubusercontent.com/kingsgeocomp/geocomputation/master/Practical-4-Functions%2C%20Packages%20and%20Methods.ipynb) we came close to inventing something a lot like pandas from scratch. 

So you already _know_ what's going on, or at least have an analogy that you can use to make sense of it. Pandas takes a column-view of data in the same way that our Dictionary-of-Lists did, it's just got a lot more features. That's why the documentation is so much more forbidding and why pandas is so much more powerful.

But at its heart, a pandas data frame ('df' for short) is just a collection of data series (i.e. columns) with an index. Each Series is like one of our column-lists from the last notebook. And the df is like the overarching dictionary that held the collection of data series (serieses?) together. OK? You've seen this before.

Let's try it with last week's data!

In [ ]:
import pandas as pd
df = pd.read_csv('http://www.reades.com/CitiesWithWikipediaData.csv')

df.head()

Check it out!

Instead of having to write a 'readRemoteCSV' function and then manually create a Dictionary-of-Lists from that remote file, we just told pandas to read it for us and it automagically converted it to a data structure that we could view. You'll notice that it even figured out where the column names were. 

All we did with `df.head()` was to ask it to print out the first 5 rows of data. If we wanted to only see the first two rows it would be `df.head(2)`. This is pretty handy, right? 

Also, it deliberately mimics the Unix command-line tool `head` (i.e. `head -5 CitiesWithWikipediaData.csv`). So you've learned two tools for the price of one!

Let's try a few more things:

In [ ]:
df.describe()

You'll probably have seen a fairly prominent warning ("Invalid value encountered in percentile"), and if you look closely you'll see that there are some fields that report things like '`NaN`' in some of the rows. These are related, but let's take a step back for a second: by calling the `describe` we were able to produce a 7-figure summary for _most_ of the columns in the data! That's a pretty handy way to summarise what's in there, right?

So, just by calling `describe`...
1. We've asked Python to describe the data frame and it has returned a set of columns with descriptive metrics for each.
2. Note what is _missing_ from this list: where are 'Name', 'MetroArea', and a couple of the other columns? Can you think why they weren't reported in the descriptives?
3. For the other columns, notice the `NaN`; these are short-hand for 'Not a Number' and it flags up a potential problem. When we are dealing with numeric columns NaNs are an issue because it's hard to know what do: is something that isn't a number something that should be ignored? Is it a major problem? Is it a 'we don't know the value' or 'we couldn't read the value'? Those are different problems!

Of course, maybe you don't want the report for all columns, maybe you're just interested in one column:

In [ ]:
print df.Population.describe()

So now we have the same information, but only for the Population column. We have to do this a _little_ differently because describing the DataFrame does some clever formatting when you're using Jupyter notebooks, and describing a Series requires us to print out the result. Also notice that `dtype` at the end: that tells us the _data type_ is a 64-bit float. You can have strings, floats, integers, booleans, etc. in a DataFrame.

But the really crucial thing is that this introduces _one_ of the two ways that we access a Series in pandas: `<data frame>.<series name>.method`. So we could get similar information on the Name column with:
```python
df.Name.describe()
```
And so forth.

In [ ]:
print df.Name.describe()
print " "
print "The mean is: " + str(df.Population.mean())

Notice that describing a text column gives us an 'object' data type because a String is a complex object, not a simple float or int.

And notice to that we can ask the df directly for a derived variable (such as the mean) just by asking the Series to do the work for us: `<data frame>.<series>.method()`. You might want to have a [look at the documentation](http://pandas.pydata.org/pandas-docs/stable/api.html#series) to see what other methods are available for a data series. It's rather a long list.

# Data Series & Indexing

A DataFrame is composed of one or more data series (columns) objects and an index that is a non-data column useful for finding individual observations. In our 'city data' data set, the index would be the city names themselves because the names _aren't_ data in the usual sense: you can't calculate a mean from them and they aren't categorical variables (e.g. 'Metro' vs 'Town') that we'd use for grouping. They are unique non-data values, so that's your index.

## Creating your own index

Ordinarily, the data making up a df are read directly from a file and the index is automatically built using the first available 'index-like' column in the file. But you are not bound by what pandas thinks is the 'right' thing to do: you can set any column as an index, or even create one of your own!

For instance, let's say that you wanted a series containing only latitudes for British cities, you could create a new Series with this custom index as follows:
```python
myLatitudes = pd.Series(
    [7063197, 6708480, 6703134, 7538620], 
    index = ['Liverpool', 'Bristol', 'Reading', 'Glasgow']
)
```
In this case, the index is a list of cities and it would, generally, make it quite quick to look up the latitude of any of the cities listed. You are never limited to _only_ looking up values by index, but this is usually faster.

In [ ]:
import pandas as pd
myLatitudes = pd.Series(
    [7063197, 6708480, 6703134, 7538620], 
    index = ['Liverpool', 'Bristol', 'Reading', 'Glasgow']
)
print "Type of myLatitudes: "      + str(type(myLatitudes))
print "Access like a dictionary: " + str(myLatitudes['Liverpool'])
print "Access like a method: "     + str(myLatitudes.Liverpool)

myLatitudes.Bristol = '555000'

print "Updated latitude: " + str(myLatitudes.Bristol)

You'll notice that we also just accessed the df in two different ways -- understanding the strengths and weaknesses of these two approaches is really important:

1. The 'method' approach (`<df>.<series name>` and `<df>.<index name>`) makes for code that is easy to read. A good example of that would be the `df.Population.mean()` that we saw above.

2. The 'dictionary' approach (`<df>['<series name>']` and `<df>['<index value>']`) is helpful when there is potentially ambiguity about what you want Python to do (you shouldn't run into this problem very often), but it's mainly about being able to access or modify a _range_ of values... as we'll see below.

## Loc and iloc

So, what about if you wanted to select several values from the df at the same time? How do you select, say, rows in the range from 0 to 2, or select Bristol and Glasgow in one go? 

Here's how:

In [ ]:
# Access like a list
print myLatitudes.iloc[0:2]

print "\n"

# Access a range
print myLatitudes.loc['Reading':]

print "\n"

# Access non-sequential values
print myLatitudes.loc[ ['Bristol','Glasgow'] ]

A simple mnemonic for loc and iloc is that iloc is about using _integers_ (i == integers) to to help you to find something in the data frame (like working with a list), while loc is about using the index _directly_ in a list-like way.

*Note*: there is a [lot more](http://pandas.pydata.org/pandas-docs/version/0.18.1/indexing.html#selection-by-position) that you can do with this.

### A Challenge for You!

If all of this has made some kind of sense, why not spend a few minutes exploring the CSV data from last week using pandas. Try the following:

1. What's the mean population?
2. What's the standard deviation of the population?
3. What's the highest rank (i.e. smallest city) in the data set?
4. Can you figure out how to calculate a z-score using one line of pandas-enabled code?

Use the coding block below for your exploration.

# Adding a New Series

Finally, and building on everything we've seen so far, to add a new series to an existing data frame we use the dictionary-like syntax:
```python
df['NewSeriesName'] = pd.Series(...Series definition...)
``` 
See how familiar that syntax is? `df['NewSeriesName']` is _exactly_ like creating and assigning a new key/value pair to a dictionary! The only difference here is that the 'value' we store in the dictionary is a Series object, and not a simple variable (String, int, float).

# Working with Data

One of the first things that we do when working with any new data set is to familiarise ourselves with it. There are a _huge_ number of ways to do this, but there are no shortcuts to:
* Reading about the data (how it was collected, what the sample size was, etc.)
* Reviewing any accompanying metadata (data about the data, column specs, etc.)
* Looking at the data itself at the row- and column-levels
* Producing descriptive statistics 
* Visualising the data using plots 
In fact, you should use _all_ of these together to really understand where the data came from, how it was handled, and whether there are gaps or other problems. If you're wondering which comes first, I've always liked this approach: _start with a chart_. We're _not_ going to do that here because, first, I want you to get a handle on pandas itself!

For the remainder of this module we're going to be working with two types of data: data about people (Socio-economic Classifcation) and data about the environment (weather). We've selected two very different types of data on purpose:
1. Because we know that some of you have interests in the human environment, and others in the natural
2. Because these are very different types of data with very different properties
3. Because we'll see that _similar_ workflows can be used with each!
What we want to highlight is that computational approaches are _highly transferrable_ between contexts. The mean or median is not _less_ relevant in one context than another, it's just more or less appropriate as a tool for understanding the data! 

We'll see the Socio-economic Classification data next week and focus on the weather API data this week.

## Weather Data 

The UK's Met Office is a world-leading weather and climate research centre, and even if it doesn't always seem like their forecasts are very accurate that's because Britain's weather is inherently _unpredictable_. They've also done a lot of work to make their weather data widely available to people like us.

I probably don't need to say a _lot_ about weather data because you've probably been making use of forecasts for much of your life! But it's _still_ worth understanding something about how weather data is gathered and reported: many organisations operate weather stations where data on wind speed, temperature, rain, and amount of sun are collected and then transmitted to a server to be integrated into a larger data set of weather _observations_ at a national or global scale. Of course, any _one_ station might be in the 'wrong' place (somewhere shady or protected from the rain) or it might even break down, but the idea is that if you have enough of them you can collect a pretty good range of data for the country and begin to look for patterns and, potentially, make predictions.

We will be access data from the MetOffice from a couple of different locations where observations such as the ones below are collected:
* <Param name="F" units="C">Feels Like Temperature (units: degrees Celsius)
* <Param name="G" units="mph">Wind Gust (units: mph)</Param>
* <Param name="H" units="%">Screen Relative Humidity (units: percent)</Param> 
* <Param name="T" units="C">Temperature (units: degrees Celsius)</Param> 
* <Param name="V" units="">Visibility (units: km?)</Param> 
* <Param name="D" units="compass">Wind Direction (units: compass degrees)</Param>  
* <Param name="S" units="mph">Wind Speed (units: mph)</Param> 
* <Param name="U" units="">Max UV Index (units: index value)</Param> 
* <Param name="W" units="">Weather Type (units: categorical)</Param> 
* <Param name="Pp" units="%">Precipitation Probability (units: percent)</Param>

These observations are only associated with a particular station (where did we see/will we see these values?), they will also be associated with _either_ a particular time in the past (when were they collected?) or, if they're forecasts, with a particular time in the future (when do we expect to see them?). 

So although weather data might seem more 'objective' than data on social class (though for obvious reasons it turns out that both are just attempts to capture data about reality, not reality itself), it may also turn out to be very complex to store and manage beccause of the temporal element _and_ the fact that it's not just a count of one thing, each of these observations uses a very different set of units.

To really get to grips with the MetOffice API you will need to RTM (Read The Manual): http://www.metoffice.gov.uk/media/pdf/3/0/DataPoint_API_reference.pdf

----
# Getting Weather Data via an API

Because the weather is changing all the time, so is the data! And, 'worse', it's becoming obsolete: the forecast from 2 years ago isn't particularly useful to us now. *And* asking for "yesterday's weather" depends on the day that we're asking! When you have data that is always changing from minute to minute or day to day then you use an API (Application Programming Interface) to access it: the API knows that "yesterday's weather" means "work out what day it is right now and then get the weather from the day before", and it also knows that "give me the current weather from station X" means "look up station X and find the latest weather report that I've received". In other words, an API is  designed with programmatic, dynamic interaction in mind right from the start.

Helpfully, the MetOffice provides a lot of documentation about their API (I'd suggest bookmarking it): http://www.metoffice.gov.uk/datapoint/support/api-reference

This type of data requires a lot more research up front to work with, but it's very flexible once you know how to 'speak API' because you can _customise_ the API request (the thing we want to know) to obtain _only_ the data we're interested in instead of being 'stuck' with what the provider wants to give you.

## Obtaining an API Key

The first step to working with the API from the MetOffice is to obtain an API key: [do that here](http://www.metoffice.gov.uk/datapoint/API).

## Making an API Request

We then use the key as part of an API request: the process by which we _ask_ for data. We're going to show you the code and output first and then we'll talk through the steps involved. But, first, you'll need to replace "???" with the API key provided to you by the MetOffice.

In [ ]:
import json, requests

api_key   = "???" # your API key
api_url   = "http://datapoint.metoffice.gov.uk/public/data/" # base URL
obs_json  = "val/wxobs/all/json/" # observations URL
fcs_json  = "val/wxfcs/all/json/" # forecasts URL

heathrow = str(3772)  # heathrow airport weather station

payload = {'res': 'hourly', 'key': api_key}
r = requests.get(api_url + obs_json + heathrow, params=payload)

#check the call - need some proper try, except stuff here
print(r.url)

#check the output
print(r.json())

OK, now let's make sense of this:
```python
import json, requests

api_key   = "???" # your API key
api_url   = "http://datapoint.metoffice.gov.uk/public/data/" # base URL
obs_json  = "val/wxobs/all/json/" # observations URL
fcs_json  = "val/wxfcs/all/json/" # forecasts URL
```
So, first we import two new modules: one that makes requests to a web server, and one that will parse JSON responses from the server in order to turn them into something that we can use.

Then we set up some default values that will allow us to build our request to the MetOffice server. The comments help us to remember what each of these variables is.

Now let's do the actual work:
```python
heathrow = str(3772)  # heathrow airport weather station

payload = {'res': 'hourly', 'key': api_key}
r = requests.get(api_url + obs_json + heathrow, params=payload)

# Check the call - need some proper try, except stuff here
print(r.url)

# Check the output
print(r.json())
```
We want the data for Heathrow Airport: we have to request it using a unique identifier (3772 in this case) because that's easier for the computer to handle than a long, potentially ambiguous string. For instance, if you asked for 'London' what would you get? The City of London? Greater London? 

We can then assemble a URL request by combining the site name, the observations URL, and the parameters. In this case that's: the type of 'resource' (the hourly observations), and our API key.

The last two steps are just about printing out the reply... It's pretty hard to figure out what that reply means, but it's actually just a kind of dictionary. That's it. It looks like a mess, but it _is_ a dictionary and the only thing that is entirely new is the fact that every string has the letter 'u' in front of it. That 'u' means 'Unicode' and it just a special kind of string that supports accents, Chinese characters, emojis, and just about anything else that you can think of...

It might be a little easier to read if we just look at the description.

In [ ]:
pdesc = r.json()['SiteRep']['Wx']
pdat  = r.json()['SiteRep']['DV']

print(pdesc)

Notice how the above also looks a lot like a mix of Python dictionaries and lists: '{' and '['.

## Using recursion to explore data dictionaries

We've seen dictionaries-of-lists and dictionaries-of-dictionaries before! We know how these work, but they've never been very easy to work with because we had to write lots and lots of nested loops:

```python
for key1 in bigDictionary:
    for key2 in bigDictionary[key1]:
        for key3 in bigDictionary[key1][key2]:
            ... And so on ...
```

And if we have to add checks on each one of these `keys` to see if it is a list, a dictionary, or a simple float/int then this code would explode in complexity and become very, very hard to follow.

But there is another way. It's a concept called _recursion_. 

Let's imagine that we have to deal with lists-of-lists (because those are a bit simpler to think about) but we don't know in advance how many lists there are inside of each list; e.g.:
```python
myList = [
    ['Value 1',
        ['Value 1.a.i', 'Value 1.a.ii'],
        ['Value 1.b.i', 'Value 1.b.ii', 
            ['Value 1.b.ii.I', 'Value 1.b.ii.II'],
        'Value 1.c'],
    ['Value 2'],
    'Value 3'
]
```
What a nightmare! That's hard to even _read_, let alone know how to process! But recursion allows us reframe this problem as something that is _almost_ simple (it's certainly elegant): we need a function that steps through a list one element at a time and then: 
* if the element is a simple value (float, int or string) then it prints it out, 
* if the element is a list then the function _calls itself_ on the nested list! 
In other words, when our list-reading-function finds a new list _inside_ the list it is currently reading, then it calls itself and passes in the list-inside-the-list.

That explanation probably _still_ doesn't take much sense, but take a look at the code below. 

**Really, really look**:

In [ ]:
def outputList(l, depth): 
    for i in range(len(l)):
        value = l[i]
        if type(value) is list:
            outputList(value, depth+1)
        elif type(value) is dict: 
            outputDict(value, depth+1)
        else:
            print "\t" * depth + "l-Value: " + value
    print "\n"

def outputDict(d, depth):
    for key, value in d.iteritems():
        print "\t" * depth + "d-Key: " + key
        if type(value) is list:
            outputList(value, depth+1)
        elif type(value) is dict:
            outputDict(value, depth+1)
        else:
            print "\t" * depth + "  d-Value: " + value
    print "\n"

So, `outputList` takes a list `l` and then steps through each element of that list. If it encounters an element that is a list, it calls `outputList` and passes it the list that it just found. If it encounters an element that is a dictionary, it calls `outputDict` and passes it the dictionary that it just found. If it encounters a simple value (the `else`) then it just prints it out.

`outputDict` works the same way.

Now, what's going on with `depth`? That variable is the one that demonstrates actual recursion. You can see that we output `"\t" * depth` as part of our print statement; that will print out `depth` tab spaces. You'll also notice that every time we recurse (call `outputList` or `outputDict` _again_) that we increment (increase) depth by 1. So this helps us to make the formatting legible so that we can see where each embedded list or dictionary actually sits within the data.

It's probably better that we just see hi it action... In our case we know that we're starting with a dictionary so we would ask `outputDict` to start outputting the content of `pdesc` (the rePly DESCription). `outputDict` then takes each of the key/value pairs in turn, looks at the value to see if _it_ is a dictionary or list or (by default) string and takes appropriate action. Don't get too stressed out if it doesn't make sense just yet, but it's such a powerful concept that it's definitely worth getting to grips with it.

In [ ]:
outputDict(pdesc, 0)

OK, so what we have is:

* A dictionary saved in the variable `pdesc` (parameter-description)
* It contains one key only: `Param`
* `pdesc['Param']` is a list of dictionaries

How do I know this? I investigated...

In [ ]:
print("Type for pdesc['Param']: " + str(type(pdesc['Param'])))

print("Type for pdesc['Param'][0]: " + str(type(pdesc['Param'][0])))

print("Contents of pdesc['Param'][0]: " + str(pdesc['Param'][0]))

So the point here is that we know have little bundles of information about the data the MetOffice is giving back to us: the parameter description dictionary tells us, for instance, that the name 'G' in the data-part of the reply is data about 'wind gusts' given in miles per hour ('mph'). We can do the same for every other parameter.

Now, let's see what we get when we look at the reply:

In [ ]:
outputDict(pdat, 0)

Right, so that's a lot more complex isn't it? But we can make sense of it in the same incremental way...

We can start off by noticing that there are some useful _generic_ fields:

* `pdat['dataDate']` will give us the date and time of the data in the reply.
* `pdat['type']` tells us that we're looking at _Obs_-ervations

And so on. The really interesting one in there is the 'Location'... let's investigate:

In [ ]:
print(pdat['Location']['name'])
print(pdat['Location']['elevation'])
print(pdat['Location']['lat'])
print(pdat['Location']['lon'])

That leaves us with the rather nasty-looking `pdat['Location']['Period']`:

In [ ]:
pdat['Location']['Period']

Again, however, if we don't panic then we can make sense of it! First, let's look at the big pieces:

* It's pretty obvious that there's a set of dictionaries in there -- we can see the '{...}'!
* We can also see things that look like readings: 'D', 'Dp', 'H', 'P'...
* We can also see two rather useful-looking bits of information: something that says 'Day' and something that looks like a timestamp (e.g. '2016-10-14Z')

Let's work on this some more by trial-and-error, starting from the point that the data structure must be a list because of the '[...]':

In [ ]:
outputDict(pdat['Location']['Period'][0], 0)

In [ ]:
outputDict(pdat['Location']['Period'][1], 0)

OK, now we know that `pdat['Location']['Period']` is a list of daily reports. How do I know that? Because when I asked for the first item in the list I got an answer with yesterday's date, and when I asked for the second item in the list I got something containing today's date! And _within_ each of those is _another_ list that contains a set of reports about the weather at Heathrow!

The _last_ clue in there is that one of the parameters is changing in an unusal way: we can guess what H (Humidity), P (Pressure) and most of the rest are from having output `pdesc` above, but the '$' is always in multiples of 60. Can you guess why?

Let's see if we can turn this into something useful... Fix the '???' so that prints out the temperature reading at Heathrow.

In [ ]:
for d in pdat['Location']['Period']: # d is short for day
    print("Date: " + d['value'])
    for i in d['Rep']: # i is short for time interval
        print("\tTime: " + str(i['$']))
        print("\t\tTemperature is: " + str(i[???]))
        print("\t\tHumidity is:    " + str(i[???]))

Can you explain why there are two days in there and why the '$' values don't overal?

Use the coding area below to print out the other values over the same period of time...

# Turning API data into a Pandas DataFrame

I've done a little searching online and no one has posted code to do this for us, so we'll have to put together everything that we learned in the past few weeks _as well as_ some new ideas about how to deal with new types of data... I am not expecting you to be able to make sense of everything that happens in the next few blocks on the first go, but I _am_ expecting you to try and I am also expecting you to be able to make use of this in the future...

First, we need to know how to work with dates and times. 

Naturally, Python has a library that can help with that so let's first have a look at the obvious date in there: `dataDate`.

In [ ]:
from datetime import datetime, timedelta 

# Ignore the time part of 'dataDate' as we're 
# getting API data with values in minutes after
# midnight. Given that, we want to set this 
# starting point to 00:00:00Z
obsDate = datetime.strptime(pdat['dataDate'].split("T")[0],'%Y-%m-%d')

print(obsDate)
print(type(obsDate)) # It's a new type of object... beyond float, int, etc.

OK, that's a start, now let's see if we can do two more things:

1. Do 'date math' so that we can add the number of minutes since midnight associated with the observation to the datetime object extracted from `dataDate`.
2. Print out _one_ observation from the full range.

We do this as a 'nested' loop: we know that we have one 'period' for each day contained in the data. And we know that within each day we have one 'report' for each hour. We can't (and don't want to) go through these in a random order so we use two `for` loops:

```python
for d in <day>:
    for h in <hour>:
        ...do something...
```

In [ ]:
for d in pdat['Location']['Period']: # d is for day of observations
    
    dataDate = datetime.strptime(d['value'],'%Y-%m-%dZ') # Convert date to datetime object
    print("Observation date: " + str(dataDate)) # Print for debugging
    
    for h in d['Rep']: # h is for hourly reports
        obsDate = dataDate + timedelta(minutes = int(h['$'])) # Add the time in minutes to the datetime
        # print(obsDate) # Debug!
        
        print("Temperature at " + str(obsDate) + " is " + str(h['T']))

See how that worked? We did the following:

1. For each day `d` in the data...
2. We used the `value` parameter to initialise (i.e. set) a variable called `dataDate`
3. We then retrieved the 'hours since midnight' from the \$ parameter and added it to dataDate using the `timedelta` function.
4. And then we found the temperature using the `T` key.

## Making a Function to Handle API Data

Let's try to moving this into a more generic function...

In [ ]:
def processMetOfficeObservations(loc): 
    """
    Process a series of 'reports' for a single
    location using the datetime object as the 
    reference time against which to build the 
    timedelta (i.e. we start from midnight and 
    the timedelta is the number of minutes past 
    midnight)
    """
    observations = {}
    
    for d in loc['Period']: # d for day
        dt = datetime.strptime(d['value'],'%Y-%m-%dZ') # Convert date to datetime object
    
        # Now deal with the actual observations (i.e. 'Reports')
        for report in d['Rep']:
            
            # Find the timestampe and add it to the date
            minutes_after_midnight = int(report['$'])
            ts = dt + timedelta(minutes=minutes_after_midnight)
            
            # For each of the possible values, set a default value
            # if the weather station doesn't actually collect that
            # parameter... can you see a problem with our defaults?
            if 'ts' not in observations:
                observations['ts'] = []
            observations['ts'].append( str(ts) )
            for key in ['D','Pt']:
                if key not in report:
                    report[key] = u""
                if key not in observations:
                    observations[key] = []
                observations[key].append(report[key])
            for key in ['W','V','S','G']:
                if key not in report or report[key] == "":
                    report[key] = 0
                if key not in observations:
                    observations[key] = []
                observations[key].append(report[key])
            for key in ['T','Dp','H']:
                if key not in report or report[key] == "":
                    report[key] = 0.0
                if key not in observations:
                    observations[key] = []
                observations[key].append(report[key])
            
    return observations

data = processMetOfficeObservations(pdat['Location'])
outputDict(data, 0)

And now this should be looking rather familiar to you... perhaps?

## Creating a DataFrame from a Dictionary

We have a dictionary-of-lists (DoL), which is exactly what we need to work with in pandas! The last step here is to figure out how to create a new data frame from this data.

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(data) # Well that was tricky...

In [ ]:
df.head() # Check that it did what we expected

Before we can do the thing that I _really_ want you to be able to do by the end of the day, we have a few more steps:

1. To rename the columns to something a little more useful.
2. To turn the 'ts' field into an _actual_ timeseries so that pandas understands what it is.
3. To convert all of the other series to the right numerical/categorical format.

Let's do this in several stages... remember that we printed out the column names earlier.

In [ ]:
# D  = Wind Direction
# Dp = Dew Point
# G  = Wind Gust
# H  = Humidity
# Pt = Pressure Tendency
# S  = Wind Speed
# T  = Temperature
# V  = Visibility
# W  = Weather Type
# ts = Time of Day

# Given this, how would you set new, easier to read column names
df.columns = ['WindDirection','DewPoint','WindGust','Humidity','PressureTendency','WindSpeed','Temperature','Visibility','WeatherType','ts']

In [ ]:
df.head()

That's looking a lot more useful, but now we need to make sure that 'ts' is treated as a time series... again, Google is your friend here: 'pandas convert datetime to time series'. 

What do you think needs to replace the '???'?

In [ ]:
df['Time'] = pd.to_datetime(???, infer_datetime_format=True)

In [ ]:
df.describe() # A quick check

## Changing column types

You'll notice that the description of the columns doesn't seem much like what we had before -- shouldn't we get the 7-figure summary of the numeric columns? The problem is that pandas didn't know what we expected the columns to be, so it's treated them all as 'objects' and not as basic numeric data types hwere possible.

So we need to fix that now... there's a function called `'astype'` that allows to convert between data types where it's fairly easy for pandas to figure out what we want to do:

In [ ]:
for c in ['WindDirection','WeatherType','PressureTendency']:
    df[c] = df[c].astype('category')
for c in ['DewPoint','Humidity','Temperature']:
    df[c] = df[c].astype('float')
for c in ['WindGust','Visibility']:
    df[c] = df[c].astype('int')
del df['ts'] # And delete a column!

In [ ]:
df.describe()

That's more like it!

# Plotting!

This has been a long, slow build towards something more exciting: plotting! In a way, this has been a lot of effort just to make a graph, but let's recognise where we're at:

* We can request data for _any_ lcoation in Britain by changing the location id.
* We can get new data _any_ time we feel like it.
* We can (in a minute) create a plot of that data.
* We can update it continuously in the future!

That's pretty awesome, right?

In [1]:
# This command tells Jupyter that we want 
# the plots to be shown inline (on this 
# web page). You'll always need to do this
# *once* on a notebook.
%matplotlib inline

In [ ]:
df.Humidity.plot()

In [ ]:
df.Temperature.plot()

In [ ]:
df.Temperature.plot(kind='box') # Not just line plots...

In [ ]:
df.WindDirection.plot() # Ooops.

I tend to think that that's quite enough to be coping with for one session... over to the script!